In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalizza per il singolo canale (grayscale)
])
# Scaricare e caricare il dataset di training
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

# Scaricare e caricare il dataset di test
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Creare DataLoader per iterare attraverso il dataset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [25]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # 1 solo canale per scala di grigi
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 10)  
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [26]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
def train_model(model, trainloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):  # Ciclo per le epoche
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            # Azzera i gradienti
            optimizer.zero_grad()

            # Passaggio forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Passaggio backward e ottimizzazione
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:    # Stampa ogni 100 batch
                print(f'Epoca {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
                running_loss = 0.0

    print('Addestramento completato.')

In [28]:
train_model(model, trainloader, criterion, optimizer, epochs=10)

Epoca 1, Batch 100, Loss: 0.785
Epoca 1, Batch 200, Loss: 0.478
Epoca 1, Batch 300, Loss: 0.404
Epoca 1, Batch 400, Loss: 0.395
Epoca 1, Batch 500, Loss: 0.342
Epoca 1, Batch 600, Loss: 0.346
Epoca 1, Batch 700, Loss: 0.320
Epoca 1, Batch 800, Loss: 0.324
Epoca 1, Batch 900, Loss: 0.317
Epoca 2, Batch 100, Loss: 0.279
Epoca 2, Batch 200, Loss: 0.275
Epoca 2, Batch 300, Loss: 0.249
Epoca 2, Batch 400, Loss: 0.261
Epoca 2, Batch 500, Loss: 0.268
Epoca 2, Batch 600, Loss: 0.245
Epoca 2, Batch 700, Loss: 0.254
Epoca 2, Batch 800, Loss: 0.259
Epoca 2, Batch 900, Loss: 0.236
Epoca 3, Batch 100, Loss: 0.208
Epoca 3, Batch 200, Loss: 0.193
Epoca 3, Batch 300, Loss: 0.224
Epoca 3, Batch 400, Loss: 0.210
Epoca 3, Batch 500, Loss: 0.206
Epoca 3, Batch 600, Loss: 0.208
Epoca 3, Batch 700, Loss: 0.199
Epoca 3, Batch 800, Loss: 0.208
Epoca 3, Batch 900, Loss: 0.201
Epoca 4, Batch 100, Loss: 0.167
Epoca 4, Batch 200, Loss: 0.169
Epoca 4, Batch 300, Loss: 0.176
Epoca 4, Batch 400, Loss: 0.175
Epoca 4,

In [36]:
def evaluate_model(model, testloader):
    correct = 0
    total = 0
    with torch.no_grad():  # Disattiva il calcolo dei gradienti per la valutazione
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuratezza sul set di test: {100 * correct / total:.2f}%')

evaluate_model(model, testloader)
evaluate_model(model, trainloader)

Accuratezza sul set di test: 92.00%
Accuratezza sul set di test: 98.95%


In [35]:
from PIL import Image
# Classi del Fashion MNIST
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Preprocessa l'immagine per Fashion MNIST
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # Assicura che l'immagine sia in scala di grigi
        transforms.Resize((28, 28)),  # Ridimensiona l'immagine a 28x28 pixel come Fashion MNIST
        transforms.ToTensor(),  # Converti l'immagine in tensore
        transforms.Normalize((0.5,), (0.5,))  # Normalizza l'immagine per grayscale (1 canale)
    ])
    
    img = Image.open(image_path)
    img = transform(img)
    img = img.unsqueeze(0)  # Aggiungi la dimensione batch
    return img

# Funzione per fare le previsioni e stampare le probabilità
def predict_confidence(model, image_path):
    img = preprocess_image(image_path)
    
    # Disabilita il calcolo dei gradienti per inferenza
    with torch.no_grad():
        outputs = model(img)
    
    # Applica softmax per ottenere le probabilità
    probabilities = torch.softmax(outputs, dim=1)
    
    # Converti il tensore in lista di probabilità per ciascuna classe
    probabilities_list = probabilities.squeeze().tolist()
    
    # Stampa le probabilità per ciascuna classe
    for i, class_name in enumerate(classes):
        print(f'{class_name}: {probabilities_list[i] * 100:.2f}%')

    # Predici la classe con la probabilità più alta
    predicted_class_idx = torch.argmax(probabilities).item()
    predicted_class = classes[predicted_class_idx]
    print(f'Predicted Class: {predicted_class}')

    return probabilities_list

# Esempio di utilizzo: prevedere su un'immagine esterna
image_path = 'maglietta-azzurra.jpg'  # Modifica questo percorso con l'immagine che vuoi testare
predict_confidence(model, image_path)


T-shirt/top: 0.00%
Trouser: 0.00%
Pullover: 0.00%
Dress: 0.00%
Coat: 0.00%
Sandal: 0.00%
Shirt: 0.00%
Sneaker: 0.00%
Bag: 100.00%
Ankle boot: 0.00%
Predicted Class: Bag


[3.466979741489906e-12,
 9.775405311607255e-14,
 2.9103324148718713e-16,
 8.017393803442522e-19,
 8.494779897474305e-19,
 1.0208819929029377e-17,
 4.252282213344327e-16,
 3.8835007895658874e-19,
 1.0,
 9.827704005682756e-16]